In [128]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## import the required Libraries




In [129]:
import os
import numpy as np
from sklearn.model_selection import train_test_split


from keras.models import Model
from keras.layers import Input, Embedding, SpatialDropout1D, Dropout, Conv1D, MaxPool1D, Flatten, concatenate, Dense, \
    LSTM, Bidirectional, Activation, MaxPooling1D, Add, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D, RepeatVector, \
    TimeDistributed, Permute, multiply, Lambda, add, Masking, BatchNormalization, Softmax, Reshape, ReLU, \
    ZeroPadding1D, subtract
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import tensorflow.keras.backend as K
import tensorflow as tf
from keras import backend as K, initializers, regularizers, constraints


# Import our dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from tensorflow.keras.layers import Layer, InputSpec
import numpy as np

from statistics import mode

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
%matplotlib inline
import tensorflow as tf


# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.metrics import Metric
from tensorflow.python.keras.utils import metrics_utils
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.keras.utils.generic_utils import to_list
import tensorflow_hub as hub



import re
import nltk
from nltk.corpus import stopwords

# neural
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential

#Metrics
from sklearn.metrics import balanced_accuracy_score

from sklearn.feature_extraction.text import TfidfVectorizer
from mlxtend.plotting import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier

##LOAD THE DOTSA CLOTHING DATASET


In [130]:
df2_train = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_dotsa/new_Clothing_train.csv", encoding='utf8')

In [131]:
#df2_train, df2_val = train_test_split(df_train, test_size=0.2, random_state=42)


In [132]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_dotsa/new_Clothing_test.csv", encoding='utf8')

In [133]:
df2_val = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_dotsa/new_Clothing_val.csv", encoding='utf8')

In [134]:
train1 = []
test1 = []
val1 = []

def dataset1(data , list1):
  for text, aspects, sentiments in zip(data['Text'], data['Aspect'], data['Sentiment']):
    aspect_list = aspects.split(', ')
    sentiment_list = sentiments.split(', ')

    for aspect, sentiment in zip(aspect_list, sentiment_list):
      if ", " in aspect:
        sub_aspects = aspect.split(", ")
        for sub_aspect in sub_aspects:
          list1.append({'Text': text, 'Aspect': sub_aspect, 'Sentiment': sentiment})
      else:
        list1.append({'Text': text, 'Aspect': aspect, 'Sentiment': sentiment})

  return list1
tr1 = dataset1(df2_train , train1)
te1 = dataset1(df2_test , test1)
va1 = dataset1(df2_val , val1)
# Create a new dataframe from the expanded data
df2_train = pd.DataFrame(tr1)
df2_test = pd.DataFrame(te1)
df2_val = pd.DataFrame(va1)



In [135]:
df2_train

,Text,Aspect,Sentiment
0,waist band is better than Bloch . it feels sof...,waist band,Positive
1,waist band is better than Bloch . it feels sof...,it,Positive
2,I am 5\ ' 6 and weigh 130-135lbs ( size 6 pant...,dance tights,Mixed
3,I am 5\ ' 6 and weigh 130-135lbs ( size 6 pant...,B,Positive
4,I bought these sandals for my Mom in two diffe...,sandals,Positive
...,...,...,...
937,LOOOOOOOVE these i can not live without them ....,these,Positive
938,I like the heel height and the style . I purch...,shoes--style,Positive
939,I like the heel height and the style . I purch...,shoes--heel height,Positive
940,I like the heel height and the style . I purch...,shoes,Mixed


##PREPROCESSING

In [136]:
## change the labels to integers for the training phase
df2_train['Sentiment'][df2_train["Sentiment"] == 'Negative'] = 0
df2_train['Sentiment'][df2_train["Sentiment"] == 'Positive'] = 1
df2_train['Sentiment'][df2_train["Sentiment"] == 'Mixed'] = 2



In [137]:
## change the labels to integers for the training phase
df2_test['Sentiment'][df2_test["Sentiment"] == 'Negative'] = 0
df2_test['Sentiment'][df2_test["Sentiment"] == 'Positive'] = 1
df2_test['Sentiment'][df2_test["Sentiment"] == 'Mixed'] = 2



In [138]:
## change the labels to integers for the training phase
df2_val['Sentiment'][df2_val["Sentiment"] == 'Negative'] = 0
df2_val['Sentiment'][df2_val["Sentiment"] == 'Positive'] = 1
df2_val['Sentiment'][df2_val["Sentiment"] == 'Mixed'] = 2



In [139]:
train_df = df2_train

In [140]:
## Convert polarity Values to Integers
train_df["Sentiment"] = pd.to_numeric(train_df["Sentiment"],errors='coerce')


In [141]:
## Split the Dataset into Training and Testing Datasets
#X_train, y_train= train_test_split(train_df,train_df['polarity'],stratify=train_df['polarity'])

X_train  = train_df
y_train = train_df['Sentiment']

In [142]:
X_test = df2_test

In [143]:
## Convert polarity Values to Integers
X_test["Sentiment"] = pd.to_numeric(X_test["Sentiment"],errors='coerce')


In [144]:
y_test = X_test["Sentiment"]

In [145]:
X_val = df2_val

In [146]:
## Convert polarity Values to Integers
X_val["Sentiment"] = pd.to_numeric(X_val["Sentiment"],errors='coerce')


In [147]:
y_val = X_val["Sentiment"]

In [148]:
NUM_WORDS = 100000 ## MAx of words to keep, based on word frequency.
EMBEDDING_SIZE = 128 ## the length of the Vector the will

## Embedding code


In [149]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.Text))
text_X_train_tokenized = tokenizer.texts_to_sequences(X_train.Text) # list of tokenized sentences
Aspect_X_train_tokenized = tokenizer.texts_to_sequences(X_train['Aspect']) # list of tokenized sentences
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.Text) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['Aspect']) # list of tokenized sentences
text_X_val_tokenized = tokenizer.texts_to_sequences(X_val.Text) # list of tokenized sentences
Aspect_X_val_tokenized = tokenizer.texts_to_sequences(X_val['Aspect']) # list of tokenized sentences

In [150]:
##Maximum length of sentences
Max_Len =  max([len(one_title) for one_title in text_X_train_tokenized])
Max_Len

486

In [151]:
## Pads sequences(or sentences) to the same length.


text_X_train_padded = pad_sequences(text_X_train_tokenized, maxlen=Max_Len)
aspect_X_train_padded = pad_sequences(Aspect_X_train_tokenized, maxlen=1)

text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)

text_X_val_padded = pad_sequences(text_X_val_tokenized, maxlen=Max_Len)
aspect_X_val_padded = pad_sequences(Aspect_X_val_tokenized, maxlen=1)


In [152]:
## Concatenate the Embedding presentation of the sentences and aspects to Create training and testing dataset
train_data =  [text_X_train_padded,aspect_X_train_padded]
test_data =   [text_X_test_padded,aspect_X_test_padded]
val_data = [text_X_val_padded,aspect_X_val_padded]

In [153]:

# modified based on `https://gist.github.com/cbaziotis/7ef97ccf71cbc14366835198c09809d2`
class Attention(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
 e: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self, W_regularizer=None, u_regularizer=None, b_regularizer=None, W_constraint=None,
                 u_constraint=None, b_constraint=None, use_W=True, use_bias=False, return_self_attend=False,
                 return_attend_weight=True, **kwargs):
        self.supports_masking = True

        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.use_W = use_W
        self.use_bias = use_bias
        self.return_self_attend = return_self_attend    # whether perform self attention and return it
        self.return_attend_weight = return_attend_weight    # whether return attention weight
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        if self.use_W:
            self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),  initializer=self.init,
                                     name='{}_W'.format(self.name), regularizer=self.W_regularizer,
                                     constraint=self.W_constraint)
        if self.use_bias:
            self.b = self.add_weight(shape=(input_shape[1],), initializer='zero', name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer, constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],), initializer=self.init, name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer, constraint=self.u_constraint)

        super(Attention, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        if self.use_W:
            x = K.tanh(K.dot(x, self.W))

        ait = Attention.dot_product(x, self.u)
        if self.use_bias:
            ait += self.b

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        if self.return_self_attend:
            attend_output = K.sum(x * K.expand_dims(a), axis=1)
            if self.return_attend_weight:
                return [attend_output, a]
            else:
                return attend_output
        else:
            return a

    def compute_output_shape(self, input_shape):
        if self.return_self_attend:
            if self.return_attend_weight:
                return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[1])]
            else:
                return input_shape[0], input_shape[-1]
        else:
            return input_shape[0], input_shape[1]

    @staticmethod
    def dot_product(x, kernel):
        """
        Wrapper for dot product operation, in order to be compatible with both
        Theano and Tensorflow
        Args:
            x (): input
            kernel (): weights
        Returns:
        """
        if K.backend() == 'tensorflow':
            return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
        else:
            return K.dot(x, kernel)

In [154]:
# attention-based lstm with aspect embedding
def atae_lstm(lstm_units = 128):
    input_text = Input(shape=(Max_Len,))
    input_aspect = Input(shape=(1,),)

    word_embedding = Embedding(NUM_WORDS, EMBEDDING_SIZE, input_length=Max_Len)
    text_embed = SpatialDropout1D(0.2)(word_embedding(input_text))

    asp_embedding = Embedding(NUM_WORDS, EMBEDDING_SIZE, input_length=Max_Len)

    aspect_embed = asp_embedding(input_aspect)
    aspect_embed = Flatten()(aspect_embed)  # reshape to 2d
    repeat_aspect = RepeatVector(Max_Len)(aspect_embed)  # repeat aspect for every word in sequence

    input_concat = concatenate([text_embed, repeat_aspect], axis=-1)
    print(input_concat.shape)
    hidden_vecs, state_h, _ = LSTM(lstm_units, return_sequences=True, return_state=True)(input_concat)
    concat = concatenate([hidden_vecs, repeat_aspect], axis=-1)

    # apply attention mechanism
    attend_weight = Attention()(concat)
    attend_weight_expand = Lambda(lambda x: K.expand_dims(x))(attend_weight)
    attend_hidden = multiply([hidden_vecs, attend_weight_expand])
    attend_hidden = Lambda(lambda x: K.sum(x, axis=1))(attend_hidden)

    attend_hidden_dense = Dense(lstm_units)(attend_hidden)
    last_hidden_dense = Dense(lstm_units)(state_h)
    final_output = Activation('tanh')(add([attend_hidden_dense, last_hidden_dense]))
    output = Dense(3, activation='softmax')(final_output)
    return Model([input_text, input_aspect], output)

In [155]:

atae_lstm_model =  atae_lstm()
atae_lstm_model.summary()

(None, 486, 256)
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 486)]        0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, 1, 128)       12800000    ['input_6[0][0]']                
                                                                                                  
 embedding_4 (Embedding)        (None, 486, 128)     12800000    ['input_5[0][0]']                
                                                                           

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [156]:

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                                    patience = 2,
                                                    verbose=1,
                                                    factor=0.1,
                                                    min_lr=0.000001)

atae_lstm_model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
atae_lstm_model.fit(x = train_data, y = y_train, validation_data = (val_data, y_val), batch_size=16, epochs=9, callbacks = [learning_rate_reduction])

Epoch 1/9
59/59 [==============================] - 18s 255ms/step - loss: 0.7654 - accuracy: 0.6826 - val_loss: 0.7548 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 2/9
59/59 [==============================] - 12s 211ms/step - loss: 0.4949 - accuracy: 0.8025 - val_loss: 0.7865 - val_accuracy: 0.6328 - lr: 0.0010
Epoch 3/9
59/59 [==============================] - ETA: 0s - loss: 0.3459 - accuracy: 0.8631
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
59/59 [==============================] - 9s 163ms/step - loss: 0.3459 - accuracy: 0.8631 - val_loss: 0.8569 - val_accuracy: 0.6384 - lr: 0.0010
Epoch 4/9
59/59 [==============================] - 8s 143ms/step - loss: 0.2063 - accuracy: 0.9363 - val_loss: 0.9582 - val_accuracy: 0.6497 - lr: 1.0000e-04
Epoch 5/9
59/59 [==============================] - ETA: 0s - loss: 0.1711 - accuracy: 0.9437
Epoch 5: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
59/59 [==============================] - 6s 107m

In [157]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
7/7 [==============================] - 0s 20ms/step - loss: 1.0257 - accuracy: 0.6633
test loss, test acc: [1.0256835222244263, 0.6632652878761292]
Accuracy of the model is -  66.32652878761292 %


##CROSS VALIDATION

##CROSS VALIDATE ON SEMEVAL RESTAURANT

In [158]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_SemEval2016/restaurants_test_single.csv", encoding='utf8')

In [159]:
df2_test

,sentence Id,sentence,aspect_category,aspect_term,from,to,polarity
0,en_BlueRibbonSushi_478218171:0,Yum!,FOOD#QUALITY,NaN,0,0,positive
1,en_BlueRibbonSushi_478218171:1,Serves really good sushi.,FOOD#QUALITY,sushi,19,24,positive
2,en_BlueRibbonSushi_478218171:2,Not the biggest portions but adequate.,FOOD#STYLE_OPTIONS,portions,16,24,neutral
3,en_BlueRibbonSushi_478218171:3,Green Tea creme brulee is a must!,FOOD#QUALITY,Green Tea creme brulee,0,22,positive
4,en_BlueRibbonSushi_478218171:4,Don't leave the restaurant without it.,FOOD#QUALITY,NaN,0,0,positive
...,...,...,...,...,...,...,...
854,en_Ray'sBoathouse_477775918:13,"All considered, I have to say that Ray's Boath...",RESTAURANT#GENERAL,Ray's Boathouse,35,50,positive
855,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,SERVICE#GENERAL,server,88,94,positive
856,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,FOOD#QUALITY,food,99,103,positive
857,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,RESTAURANT#MISCELLANEOUS,NaN,0,0,negative


In [160]:
df2_test.drop(columns=['from' , 'to', 'aspect_category'], inplace=True)


In [161]:
df2_test=df2_test.dropna(axis=0)


In [162]:
df2_test

,sentence Id,sentence,aspect_term,polarity
1,en_BlueRibbonSushi_478218171:1,Serves really good sushi.,sushi,positive
2,en_BlueRibbonSushi_478218171:2,Not the biggest portions but adequate.,portions,neutral
3,en_BlueRibbonSushi_478218171:3,Green Tea creme brulee is a must!,Green Tea creme brulee,positive
6,en_BlueRibbonSushi_478218345:1,– I can't say enough about this place.,place,positive
7,en_BlueRibbonSushi_478218345:2,It has great sushi and even better service.,sushi,positive
...,...,...,...,...
852,en_Ray'sBoathouse_477775918:10,Our server continued to be attentive throughou...,Ray's,neutral
854,en_Ray'sBoathouse_477775918:13,"All considered, I have to say that Ray's Boath...",Ray's Boathouse,positive
855,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,server,positive
856,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,food,positive


In [163]:
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1
df2_test['polarity'][df2_test["polarity"] == 'neutral'] = 2


<ipython-input-163-ee27ebe34892>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
<ipython-input-163-ee27ebe34892>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1
<ipython-input-163-ee27ebe34892>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'neutral'] = 2


In [164]:
X_test = df2_test

In [165]:

## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


<ipython-input-165-9dad12974081>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [166]:
y_test = X_test["polarity"]

In [167]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.Text))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_term']) # list of tokenized sentences


In [168]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [169]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [170]:

print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")


Evaluate on test data
21/21 [==============================] - 0s 16ms/step - loss: 1.2461 - accuracy: 0.7477
test loss, test acc: [1.2460598945617676, 0.7476922869682312]
Accuracy of the model is -  74.76922869682312 %


##CROSS VALIDATE ON YASO

In [171]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_YASO/TSA-MD_val_single.csv")

In [172]:
def delete_rows_with_target_not_found(df):
    return df.loc[df['aspect_term'] != 'Target Not Found']


In [173]:
df2_test = delete_rows_with_target_not_found(df2_test)

In [174]:
df2_test

,sentence,aspect_term,begin,end,polarity
0,"""Typical"" Italian food was far from typical.",Italian food,10,22,negative
1,After a few weeks the tablet started asking fo...,sound,128,133,positive
2,After a few weeks the tablet started asking fo...,buy,251,254,positive
3,After a few weeks the tablet started asking fo...,wireless headphones,202,221,positive
4,After a few weeks the tablet started asking fo...,resolution of the camera is decent,88,122,positive
...,...,...,...,...,...
306,"yesterday, I ordered four chicken sandwiches b...",delivered,105,114,negative
307,"yesterday, I ordered four chicken sandwiches b...",quality of food,53,68,negative
308,"yesterday, I ordered four chicken sandwiches b...",chicken sandwiches,26,44,negative
309,you can find there anything you want .,there,13,18,positive


In [175]:
df2_test

,sentence,aspect_term,begin,end,polarity
0,"""Typical"" Italian food was far from typical.",Italian food,10,22,negative
1,After a few weeks the tablet started asking fo...,sound,128,133,positive
2,After a few weeks the tablet started asking fo...,buy,251,254,positive
3,After a few weeks the tablet started asking fo...,wireless headphones,202,221,positive
4,After a few weeks the tablet started asking fo...,resolution of the camera is decent,88,122,positive
...,...,...,...,...,...
306,"yesterday, I ordered four chicken sandwiches b...",delivered,105,114,negative
307,"yesterday, I ordered four chicken sandwiches b...",quality of food,53,68,negative
308,"yesterday, I ordered four chicken sandwiches b...",chicken sandwiches,26,44,negative
309,you can find there anything you want .,there,13,18,positive


In [176]:
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1



<ipython-input-176-5dfe2b17fbcd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1


In [177]:
X_test = df2_test

In [178]:
## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [179]:
y_test = X_test["polarity"]

In [180]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.Text))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_term']) # list of tokenized sentences


In [181]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [182]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [183]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
10/10 [==============================] - 0s 12ms/step - loss: 0.9648 - accuracy: 0.7749
test loss, test acc: [0.9648275375366211, 0.7749196290969849]
Accuracy of the model is -  77.49196290969849 %


##CROSS VALIDATE ON MAMS

In [184]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_MAMS/ATSA_test_single.csv", encoding='utf8')

In [185]:
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1
df2_test['polarity'][df2_test["polarity"] == 'neutral'] = 2

In [186]:
df2_test

,sentence,aspect_terms,polarity
0,The food was served promptly but the meal wasn...,food,2
1,The food was served promptly but the meal wasn...,served,1
2,The food was served promptly but the meal wasn...,appetizers,1
3,"When I got home, there was a message on the ma...",owner,2
4,"When I got home, there was a message on the ma...",waitress,0
...,...,...,...
1331,Pair you food with the excellent beers on tap ...,priced wine list,1
1332,"They were out of Hennessy, Alize, Mojitos, Pin...",Mojitos,2
1333,"They were out of Hennessy, Alize, Mojitos, Pin...",Pinot,2
1334,"They were out of Hennessy, Alize, Mojitos, Pin...",drink list,0


In [187]:
X_test = df2_test

In [188]:
## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [189]:
y_test = X_test["polarity"]

In [190]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.Text))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_terms']) # list of tokenized sentences


In [191]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [192]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [193]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
42/42 [==============================] - 0s 11ms/step - loss: 3.3415 - accuracy: 0.3076
test loss, test acc: [3.3414506912231445, 0.30763474106788635]
Accuracy of the model is -  30.763474106788635 %


##CROSS VALIDATE ON SENTIHOOD DATASET


In [194]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_Sentihood/sentihood_test_single.csv", encoding='utf8')

In [195]:

df2_test.drop(columns=['to' , 'from'], inplace=True)

In [196]:
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1


<ipython-input-196-27e789cd0644>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
<ipython-input-196-27e789cd0644>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1


In [197]:
X_test = df2_test

In [198]:
## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [199]:
y_test = X_test["polarity"]

In [200]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.Text))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_term']) # list of tokenized sentences


In [201]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [202]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [203]:

print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")


Evaluate on test data
53/53 [==============================] - 1s 9ms/step - loss: 1.4650 - accuracy: 0.7242
test loss, test acc: [1.4649771451950073, 0.7242406010627747]
Accuracy of the model is -  72.42406010627747 %


##CROSS VALIDATE ON DOTSA BOOKS

In [204]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_dotsa/new_Books_test.csv", encoding='utf8')

In [205]:
train1 = []
test1 = []
val1 = []

def dataset1(data , list1):
  for text, aspects, sentiments in zip(data['Text'], data['Aspect'], data['Sentiment']):
    aspect_list = aspects.split(', ')
    sentiment_list = sentiments.split(', ')

    for aspect, sentiment in zip(aspect_list, sentiment_list):
      if ", " in aspect:
        sub_aspects = aspect.split(", ")
        for sub_aspect in sub_aspects:
          list1.append({'Text': text, 'Aspect': sub_aspect, 'Sentiment': sentiment})
      else:
        list1.append({'Text': text, 'Aspect': aspect, 'Sentiment': sentiment})

  return list1

te1 = dataset1(df2_test , test1)

# Create a new dataframe from the expanded data

test_df = pd.DataFrame(te1)




In [206]:
## change the labels to integers for the training phase
test_df['Sentiment'][test_df["Sentiment"] == 'Negative'] = 0
test_df['Sentiment'][test_df["Sentiment"] == 'Positive'] = 1
test_df['Sentiment'][test_df["Sentiment"] == 'Mixed'] = 2



In [207]:
test_df

,Text,Aspect,Sentiment
0,This story is one of too many heartbreaking st...,story,2
1,Good Faye Kellerman ! ! ! She manages to keep ...,Faye Kellerman,1
2,Good Faye Kellerman ! ! ! She manages to keep ...,characters,1
3,Good Faye Kellerman ! ! ! She manages to keep ...,it,1
4,Good book on cancer -- a little longer than I ...,book,1
...,...,...,...
363,This was my first experience reading a Faye Ke...,it,2
364,This was my first experience reading a Faye Ke...,it--story,1
365,This was my first experience reading a Faye Ke...,it--plot,0
366,This was my first experience reading a Faye Ke...,it--characters,1


In [208]:
X_test = test_df

In [209]:
## Convert polarity Values to Integers
X_test["Sentiment"] = pd.to_numeric(X_test["Sentiment"],errors='coerce')


In [210]:
y_test = X_test["Sentiment"]

In [211]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.Text))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.Text) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['Aspect']) # list of tokenized sentences


In [212]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [213]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [214]:

print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")


Evaluate on test data
12/12 [==============================] - 0s 10ms/step - loss: 1.3697 - accuracy: 0.6141
test loss, test acc: [1.3697479963302612, 0.614130437374115]
Accuracy of the model is -  61.4130437374115 %


##CROSS VALIDATE ON DOTSA CLOTHING

In [215]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_dotsa/new_Clothing_test.csv", encoding='utf8')

In [216]:
train1 = []
test1 = []
val1 = []

def dataset1(data , list1):
  for text, aspects, sentiments in zip(data['Text'], data['Aspect'], data['Sentiment']):
    aspect_list = aspects.split(', ')
    sentiment_list = sentiments.split(', ')

    for aspect, sentiment in zip(aspect_list, sentiment_list):
      if ", " in aspect:
        sub_aspects = aspect.split(", ")
        for sub_aspect in sub_aspects:
          list1.append({'Text': text, 'Aspect': sub_aspect, 'Sentiment': sentiment})
      else:
        list1.append({'Text': text, 'Aspect': aspect, 'Sentiment': sentiment})

  return list1

te1 = dataset1(df2_test , test1)

# Create a new dataframe from the expanded data

test_df = pd.DataFrame(te1)




In [217]:
## change the labels to integers for the training phase
test_df['Sentiment'][test_df["Sentiment"] == 'Negative'] = 0
test_df['Sentiment'][test_df["Sentiment"] == 'Positive'] = 1
test_df['Sentiment'][test_df["Sentiment"] == 'Mixed'] = 2



In [218]:
X_test = test_df

In [219]:
## Convert polarity Values to Integers
X_test["Sentiment"] = pd.to_numeric(X_test["Sentiment"],errors='coerce')


In [220]:
y_test = X_test["Sentiment"]

In [221]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.Text))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.Text) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['Aspect']) # list of tokenized sentences


In [222]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [223]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [224]:
"""
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")
"""

'\nprint("Evaluate on test data")\nresults = atae_lstm_model.evaluate(x = test_data , y = y_test )\nprint("test loss, test acc:", results)\nprint("Accuracy of the model is - " , results[1]*100 , "%")\n'

##CROSS VALIDATE ON DOTSA HOTELS

In [225]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_dotsa/new_Hotels_test.csv", encoding='utf8')

In [226]:
train1 = []
test1 = []
val1 = []

def dataset1(data , list1):
  for text, aspects, sentiments in zip(data['Text'], data['Aspect'], data['Sentiment']):
    aspect_list = aspects.split(', ')
    sentiment_list = sentiments.split(', ')

    for aspect, sentiment in zip(aspect_list, sentiment_list):
      if ", " in aspect:
        sub_aspects = aspect.split(", ")
        for sub_aspect in sub_aspects:
          list1.append({'Text': text, 'Aspect': sub_aspect, 'Sentiment': sentiment})
      else:
        list1.append({'Text': text, 'Aspect': aspect, 'Sentiment': sentiment})

  return list1

te1 = dataset1(df2_test , test1)

# Create a new dataframe from the expanded data

test_df = pd.DataFrame(te1)




In [227]:
## change the labels to integers for the training phase
test_df['Sentiment'][test_df["Sentiment"] == 'Negative'] = 0
test_df['Sentiment'][test_df["Sentiment"] == 'Positive'] = 1
test_df['Sentiment'][test_df["Sentiment"] == 'Mixed'] = 2



In [228]:
X_test = test_df

In [229]:
## Convert polarity Values to Integers
X_test["Sentiment"] = pd.to_numeric(X_test["Sentiment"],errors='coerce')


In [230]:
y_test = X_test["Sentiment"]

In [231]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.Text))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.Text) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['Aspect']) # list of tokenized sentences


In [232]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [233]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [234]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
18/18 [==============================] - 0s 12ms/step - loss: 0.5039 - accuracy: 0.8670
test loss, test acc: [0.5038502812385559, 0.8670309782028198]
Accuracy of the model is -  86.70309782028198 %


##CROSS VALIDATE ON DOTSA RESTAURANTS

In [235]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_dotsa/new_Restaurant_test.csv", encoding='utf8')

In [236]:
train1 = []
test1 = []
val1 = []

def dataset1(data , list1):
  for text, aspects, sentiments in zip(data['Text'], data['Aspect'], data['Sentiment']):
    aspect_list = aspects.split(', ')
    sentiment_list = sentiments.split(', ')

    for aspect, sentiment in zip(aspect_list, sentiment_list):
      if ", " in aspect:
        sub_aspects = aspect.split(", ")
        for sub_aspect in sub_aspects:
          list1.append({'Text': text, 'Aspect': sub_aspect, 'Sentiment': sentiment})
      else:
        list1.append({'Text': text, 'Aspect': aspect, 'Sentiment': sentiment})

  return list1

te1 = dataset1(df2_test , test1)

# Create a new dataframe from the expanded data

test_df = pd.DataFrame(te1)




In [237]:
## change the labels to integers for the training phase
test_df['Sentiment'][test_df["Sentiment"] == 'Negative'] = 0
test_df['Sentiment'][test_df["Sentiment"] == 'Positive'] = 1
test_df['Sentiment'][test_df["Sentiment"] == 'Mixed'] = 2



In [238]:
X_test = test_df

In [239]:
## Convert polarity Values to Integers
X_test["Sentiment"] = pd.to_numeric(X_test["Sentiment"],errors='coerce')


In [240]:
y_test = X_test["Sentiment"]

In [241]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.Text))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.Text) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['Aspect']) # list of tokenized sentences


In [242]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [243]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [244]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
22/22 [==============================] - 0s 11ms/step - loss: 1.0623 - accuracy: 0.6833
test loss, test acc: [1.062304139137268, 0.6832844614982605]
Accuracy of the model is -  68.32844614982605 %
